In [3]:
!pip install nibabel

In [10]:
!pip install matplotlib

In [12]:
import numpy as np
import os
import glob
import nibabel as nib
import sys
import json
import matplotlib.pyplot as plt

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path(r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\src\preprocessing.py').resolve().parent.parent))